I'd like to compare the efficiency of vectorization implementation in practice.

*(inspired by following slides...)*

<img src="https://ifh.cc/g/tr2fyq.jpg">

0. weight 및 input은 numpy library의 난수 생성 모듈을 활용함: https://numpy.org/doc/2.1/reference/random/generated/numpy.random.rand.html

In [23]:
import numpy as np

In [24]:
# weight = [np.random.rand() for _ in range(4)]
weight = np.random.rand(4)
weight

array([0.87564442, 0.34859914, 0.50793036, 0.74550979])

In [25]:
input  = np.random.rand(4)
input

array([0.11948636, 0.2314196 , 0.55878437, 0.93252264])

`np.random.rand()`

1. totally manual code.

In [26]:
result_1 = weight[0] * input[0] + weight[1] * input[1] + weight[2] * input[2] + weight[3] * input[3]
result_1

np.float64(1.1643285416291145)

2. using for-loop

In [27]:
result_2 = 0

for i in range(len(weight)):
  result_2 += weight[i] * input[i]

result_2

np.float64(1.1643285416291145)

3. using vector

In [28]:
result_3 = np.matmul(weight, input)
result_3

np.float64(1.1643285416291145)

- weighted-sum 결과 비교

In [ ]:
result_1 == result_2  # <class 'numpy.bool_'>

np.True_

In [29]:
(result_1 == result_2) and (result_2 == result_3)

np.True_

In [12]:
result_2 == result_3

np.True_

In [32]:
type(result_3)

numpy.float64

- experiment : `n = 100,000,000`

1. totally manual implementation

In [34]:
n = 100000000 # 100 M
# w = [np.random.rand() for _ in range(n)]
# x = [np.random.rand() for _ in range(n)]
w = np.random.rand(n)
x = np.random.rand(n)

len(w)

100000000

In [35]:
result_01 = 0

for i in range(len(w)):
  result_01 += w[i] * x[i]

result_01

np.float64(25002968.202735566)

In [36]:
result_02 = np.matmul(w, x)

result_02

np.float64(25002968.202728584)

- 결과 비교!!


    *   난수 생성에 0.6초
    *   순차적 연산에 14초
    *   병렬적 연산에 0.0n초 남짓 소요됨을 확인 !!



In [37]:
result_01 == result_02  

np.False_

In [38]:
int(result_01) == int(result_02)

True

- 소수점 아래 연산결과가 미세하게 다른 이유?

> 부동소수점 연산의 누적 오차 때문 : floating-point accumulation error

> 위에서도 가중합 결과의 소수점 아래 다섯째 자리부터 차이가 확인된다.

이런 차이는 **부동소수점의 유한 정밀도(IEEE 754 표준)**에서 비롯된 것으로, 일반적인 머신러닝이나 데이터 분석에서는 무시 가능한 수준이라고 한다...

In [39]:
np.allclose(result_01, result_02) # True > 오차 허용 범위 내에서 동일하다는 의미

True

`np.allclose(array1, array2)`: 두 배열이 거의 같은 값인지(수치적으로 거의 같은지를) 판단할 때 사용하는 NumPy 함수

- **결론**

차이는 계산 순서 차이로 인한 부동소수점 정밀도의 한계에서 야기된 것으로,

둘 다 수학적으로 같은 연산을 수행하고 있다.

하지만 연산 속도와 효율성 측면에서 벡터화된 계산이 명확한 이점을 지닌다 !